In [54]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json
import openai as client


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,

}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [43]:
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-turbo-preview",
#     tools=functions,
# )

assistant_id = "asst_3fWoqVkNylH5dXbozxVl6o4s"

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content":"I want to know if the Salesforce stock is a good buy"
        }
    ]
)

thread

Thread(id='thread_RJVH51ISc06Jcq3Va4IaQQpK', created_at=1712901078, metadata={}, object='thread')

In [82]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_yf1quQa05gIcViHN0gB6CcIg', assistant_id='asst_3fWoqVkNylH5dXbozxVl6o4s', cancelled_at=None, completed_at=None, created_at=1712903179, expires_at=1712903779, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_RJVH51ISc06Jcq3Va4IaQQpK', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type'

In [49]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

In [25]:
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


In [90]:
get_run(run.id, thread.id).status

'completed'

In [91]:
get_messages(thread.id)


user: I want to know if the Salesforce stock is a good buy
assistant: Analyzing whether Salesforce (CRM) stock is a good buy involves reviewing its financial statements and stock performance. Here’s a consolidated summary of the key aspects:

### Income Statement Highlights:
- **Total Revenue Growth**: There has been a steady increase in total revenue over the past few periods, indicating healthy business expansion.
- **Net Income**: There's variability in net income; however, the most recent period shows a significant net income, reflecting a strong profitability aspect.
- **Operating Expenses and Profit**: Salesforce has managed its operating expenses reasonably well, maintaining a stable operating income.

### Balance Sheet Strengths:
- **Total Assets**: The total assets have been growing, indicating expansion and possibly investment in growth opportunities.
- **Liabilities and Equity**: While liabilities have also increased, the equity position remains robust, suggesting the compan

In [76]:
def send_message(thread_id, content):
    client.beta.threads.messages.create(thread_id=thread_id, role="user", content=content)

In [66]:
def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action.id,
            }
        )
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs,
    )

In [86]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "NET"}
Calling function: get_balance_sheet with arg {"ticker": "NET"}
Calling function: get_daily_stock_performance with arg {"ticker": "NET"}


Run(id='run_yf1quQa05gIcViHN0gB6CcIg', assistant_id='asst_3fWoqVkNylH5dXbozxVl6o4s', cancelled_at=None, completed_at=None, created_at=1712903179, expires_at=1712903779, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=1712903219, status='queued', thread_id='thread_RJVH51ISc06Jcq3Va4IaQQpK', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={

In [77]:
send_message(thread.id, "Now I want to know if Cloudflare is a good buy")